<a href="https://colab.research.google.com/github/lgregs/TCC-CD-USP/blob/main/TCC_USP_get_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TCC CD USP
* LUCAS GREGORIO

## Imports e Configs

In [1]:
!pip install kagglehub --quiet

In [2]:
import os
import shutil
import kagglehub
import pandas as pd
import sqlite3
import pickle
from google.colab import files
import re

In [3]:
# Configurações do Notebook!
KAGGLE_JSON = 'kaggle.json'
KAGGLE_DIR = os.path.expanduser("~/.kaggle")
DB_FULL = 'balancos.sqlite'
DB_TOP10 = 'empresas_turquia.db'
PICKLE_FILE = 'processed_files.pkl'
TOP10_EMPRESAS = ['SAHOL', 'KCHOL', 'SISE', 'EREGL', 'FROTO', 'BIMAS', 'THYAO', 'TUPRS', 'ARCLK', 'TOASO']


## Dowload do Dataset do Kaggle

In [4]:
# 4. Upload das Credenciais Kaggle
uploaded = files.upload()
os.makedirs(KAGGLE_DIR, exist_ok=True)
shutil.move(KAGGLE_JSON, os.path.join(KAGGLE_DIR, KAGGLE_JSON))
os.chmod(os.path.join(KAGGLE_DIR, KAGGLE_JSON), 600)

Saving kaggle.json to kaggle.json


In [5]:
# 5. Baixar Dataset
path = kagglehub.dataset_download("agrafintech/turkish-public-companies-balance-sheets-from-kap")
print(f"Arquivos baixados em: {path}")

Arquivos baixados em: /kaggle/input/turkish-public-companies-balance-sheets-from-kap


In [6]:
# 6. Pasta Principal
main_folder = os.path.join(path, 'Turkish Public Companies Balance Sheets from KAP')
print("Arquivos disponíveis:", os.listdir(main_folder))

Arquivos disponíveis: ['SAYAS_2021.xlsx', 'RYSAS_2016.xlsx', 'SAHOL_2013.xlsx', 'KATMR_2013.xlsx', 'ASUZU_2023.xlsx', 'PENGD_2017.xlsx', 'KAPLM_2012.xlsx', 'FRIGO_2024.xlsx', 'TEKTU_2023.xlsx', 'BANVT_2019.xlsx', 'AKENR_2011.xlsx', 'INTEM_2014.xlsx', 'PEHOL_2010.xlsx', 'KGYO_2024.xlsx', 'PARSN_2024.xlsx', 'ALKA_2011.xlsx', 'CIMSA_2008.xlsx', 'SEYKM_2018.xlsx', 'DOGUB_2012.xlsx', 'THYAO_2014.xlsx', 'OZGYO_2017.xlsx', 'IZENR_2020.xlsx', 'TARKM_2021.xlsx', 'PAGYO_2021.xlsx', 'ISKPL_2022.xlsx', 'OZGYO_2008.xlsx', 'ESCAR_2018.xlsx', 'VESBE_2015.xlsx', 'KRGYO_2022.xlsx', 'YESIL_2023.xlsx', 'PCILT_2019.xlsx', 'ATLAS_2024.xlsx', 'DESPC_2011.xlsx', 'ORGE_2020.xlsx', 'MRGYO_2023.xlsx', 'RTALB_2012.xlsx', 'IDGYO_2018.xlsx', 'SANEL_2023.xlsx', 'A1CAP_2021.xlsx', 'BIZIM_2013.xlsx', 'BIZIM_2015.xlsx', 'ARSAN_2023.xlsx', 'MEGMT_2022.xlsx', 'YESIL_2019.xlsx', 'DCTTR_2021.xlsx', 'SURGY_2022.xlsx', 'SKTAS_2016.xlsx', 'EGGUB_2019.xlsx', 'NIBAS_2014.xlsx', 'GEREL_2013.xlsx', 'EDIP_2024.xlsx', 'YONGA_2022.

In [7]:
# 7. Compactar e disponibilizar ZIP (opcional)
shutil.make_archive('balance_sheets_dataset', 'zip', main_folder)
files.download('balance_sheets_dataset.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Processamento dos arquivos e Criação das Tabelas SQL

In [8]:
def processar_excels_e_salvar_sqlite(folder, db_path, top10_only=False):
    import re
    import pickle
    conn = sqlite3.connect(db_path)
    arquivos_excel = [f for f in os.listdir(folder) if f.endswith(('.xls', '.xlsx'))]
    todos_os_dados = []

    for arquivo in arquivos_excel:
        partes = os.path.splitext(arquivo)[0].split('_')
        empresa = partes[0]
        ano = None
        if len(partes) > 1:
            ano_raw = partes[1]
            ano_match = re.search(r'\d{4}', ano_raw)
            if ano_match:
                ano = int(ano_match.group())

        if top10_only and empresa not in TOP10_EMPRESAS:
            continue

        try:
            df = pd.read_excel(os.path.join(folder, arquivo))
            df['empresa'] = empresa
            df['ano'] = ano
            todos_os_dados.append(df)

            if not top10_only:
                nome_tabela = os.path.splitext(arquivo)[0].lower().replace(" ", "_").replace("-", "_")
                df.to_sql(nome_tabela, conn, if_exists='replace', index=False)

        except Exception as e:
            print(f"Erro ao processar {arquivo}: {e}")

    # Salvar base unificada se aplicável
    if todos_os_dados:
        df_final = pd.concat(todos_os_dados, ignore_index=True)

        if top10_only:
            df_final.to_sql('balancos_turquia', conn, if_exists='replace', index=False)
            print("Base TOP 10 criada!")

        # Save as Pickle
        nome_pickle = 'balancos_top10.pkl' if top10_only else 'balancos_completo.pkl'
        with open(nome_pickle, 'wb') as f:
            pickle.dump(df_final, f)
        print(f"Pickle '{nome_pickle}' salvo com {len(df_final)} registros.")

    else:
        print("Nenhum dado encontrado.")

    conn.close()


In [9]:
# 9. Gerar Base Completa
print("Processando todos os arquivos...")
processar_excels_e_salvar_sqlite(main_folder, DB_FULL, top10_only=False)

Processando todos os arquivos...
✅ Pickle 'balancos_completo.pkl' salvo com 915039 registros.


In [10]:
# 10. Gerar Base TOP 10 Empresas
print("Processando apenas TOP 10...")
processar_excels_e_salvar_sqlite(main_folder, DB_TOP10, top10_only=True)

Processando apenas TOP 10...
✅ Base TOP 10 criada!
✅ Pickle 'balancos_top10.pkl' salvo com 22312 registros.


In [11]:
# 11. Criar Tabela de Empresas Únicas
def criar_tabela_empresas(folder, db_path):
    conn = sqlite3.connect(db_path)
    arquivos_excel = os.listdir(folder)
    empresas = [arquivo.split("_")[0] for arquivo in arquivos_excel]
    empresas_unicas = sorted(set(empresas))

    df_empresas = pd.DataFrame({'nome_empresa': empresas_unicas})
    df_empresas.to_sql('empresas', conn, if_exists='replace', index_label='id')

    conn.close()
    print(f"✅ Tabela 'empresas' criada no banco {db_path}!")

In [12]:
# 12. Executar criação da Tabela de Empresas
criar_tabela_empresas(main_folder, DB_FULL)

✅ Tabela 'empresas' criada no banco balancos.sqlite!


In [13]:
# 13. Visualizar primeiras empresas
conn = sqlite3.connect(DB_FULL)
df_empresas = pd.read_sql_query("SELECT * FROM empresas LIMIT 10", conn)
print(df_empresas)
conn.close()

   id nome_empresa
0   0        A1CAP
1   1        ACSEL
2   2         ADEL
3   3        ADESE
4   4        ADGYO
5   5        AEFES
6   6        AFYON
7   7        AGHOL
8   8        AGROT
9   9         AGYO


In [14]:
df_empresas

,id,nome_empresa
0,0,A1CAP
1,1,ACSEL
2,2,ADEL
3,3,ADESE
4,4,ADGYO
5,5,AEFES
6,6,AFYON
7,7,AGHOL
8,8,AGROT
9,9,AGYO


In [17]:
# --- 1. Carregar o Pickle dos Top10
with open('balancos_top10.pkl', 'rb') as f:
    df_top10 = pickle.load(f)

print(f"✅ Pickle carregado: {df_top10.shape[0]} registros")

# --- 2. Mapear ID para cada empresa
empresas_unicas = sorted(df_top10['empresa'].unique())
empresa2id = {empresa: idx + 1 for idx, empresa in enumerate(empresas_unicas)}

print("Mapa empresa -> ID:")
print(empresa2id)

# --- 3. Criar coluna 'id_empresa'
df_top10['id_empresa'] = df_top10['empresa'].map(empresa2id)

# --- 4. Organizar colunas (ID primeiro)
colunas = ['id_empresa', 'empresa', 'ano'] + [col for col in df_top10.columns if col not in ['id_empresa', 'empresa', 'ano']]
df_top10 = df_top10[colunas]

# --- 5. Salvar resultado em novo Pickle (opcional)
with open('balancos_top10_id.pkl', 'wb') as f:
    pickle.dump(df_top10, f)

print("✅ Novo dataframe com ID salvo: balancos_top10_id.pkl")

# Exibir amostra
df_top10.head()

✅ Pickle carregado: 22312 registros
Mapa empresa -> ID:
{'ARCLK': 1, 'BIMAS': 2, 'EREGL': 3, 'FROTO': 4, 'KCHOL': 5, 'SAHOL': 6, 'SISE': 7, 'THYAO': 8, 'TOASO': 9, 'TUPRS': 10}
✅ Novo dataframe com ID salvo: balancos_top10_id.pkl


,id_empresa,empresa,ano,Desc,Yıllık,9 Aylık,6 Aylık,3 Aylık
0,6,SAHOL,2013,Dönen Varlıklar,1.051343e+11,1.035133e+11,9.385025e+10,9.069027e+10
1,6,SAHOL,2013,Nakit ve Nakit Benzerleri,5.566531e+09,7.556368e+09,6.758761e+09,8.811850e+09
2,6,SAHOL,2013,Finansal Yatırımlar,1.699315e+10,1.251304e+10,1.304584e+10,7.141221e+09
3,6,SAHOL,2013,Ticari Alacaklar,1.211220e+09,1.406831e+09,1.276791e+09,1.094996e+09
4,6,SAHOL,2013,Finans Sektörü Faaliyetlerinden Alacaklar,5.941694e+10,5.886337e+10,5.458826e+10,5.347631e+10


In [18]:
df_top10

,id_empresa,empresa,ano,Desc,Yıllık,9 Aylık,6 Aylık,3 Aylık
0,6,SAHOL,2013,Dönen Varlıklar,1.051343e+11,1.035133e+11,9.385025e+10,9.069027e+10
1,6,SAHOL,2013,Nakit ve Nakit Benzerleri,5.566531e+09,7.556368e+09,6.758761e+09,8.811850e+09
2,6,SAHOL,2013,Finansal Yatırımlar,1.699315e+10,1.251304e+10,1.304584e+10,7.141221e+09
3,6,SAHOL,2013,Ticari Alacaklar,1.211220e+09,1.406831e+09,1.276791e+09,1.094996e+09
4,6,SAHOL,2013,Finans Sektörü Faaliyetlerinden Alacaklar,5.941694e+10,5.886337e+10,5.458826e+10,5.347631e+10
...,...,...,...,...,...,...,...,...
22307,8,THYAO,2015,Diğer Nakit Girişi/Çıkışı,NaN,NaN,0.000000e+00,0.000000e+00
22308,8,THYAO,2015,Nakit ve Benzerlerindeki Değişim,NaN,NaN,1.179000e+09,-3.400000e+07
22309,8,THYAO,2015,Diğer Nakit ve Nakit Benzerlerindeki Artış,NaN,NaN,0.000000e+00,0.000000e+00
22310,8,THYAO,2015,Dönem Başı Nakit Değerler,NaN,NaN,1.474000e+09,1.474000e+09


In [19]:
df_top10.to_csv('balancos_top10_id.csv', index=False)